In [1]:
# Based on https://www.pyimagesearch.com/2016/08/01/lenet-convolutional-neural-network-in-python/

In [2]:
import numpy as np
import cv2
import matplotlib.pyplot as plt
import numpy as np
%matplotlib inline

def show(cv_image):
    img = cv2.cvtColor(cv_image, cv2.COLOR_BGR2RGB)
    plt.figure(figsize = (20,10))
    plt.imshow(img, interpolation='nearest')
    plt.axis('off')
    plt.show()
    
def showGrey(img):
    plt.imshow(img, cmap='gray')
    plt.axis('off')
    plt.show()

In [65]:
# imports 
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from keras.models import Sequential
from keras.layers.convolutional import Conv2D
from keras.layers.convolutional import MaxPooling2D
from keras.layers.core import Activation, Dropout, Flatten, Dense
from keras.optimizers import SGD, Adadelta
from keras.losses import categorical_crossentropy
from keras.callbacks import EarlyStopping
from keras.utils import np_utils
from imutils import paths
import numpy as np
import cv2
import os

In [60]:
# load data set

size=(28, 28)

def image_to_feature_vector(image, size=size):
    return cv2.resize(image, size)

def rotate(image, angle=90):
    rows = image.shape[0]
    cols = image.shape[1]
    M = cv2.getRotationMatrix2D((cols/2,rows/2),angle,1)
    return cv2.warpAffine(image,M,(cols,rows))
    
dataset_dir = "/Users/rokickik/Downloads/Workstation/orientation/20x/left"
imagePaths = list(paths.list_images(dataset_dir))

raw_data = []
raw_labels = []
for (i, imagePath) in enumerate(imagePaths):
    
    image = cv2.imread(imagePath)
    image = cv2.cvtColor(image, cv2.COLOR_RGB2GRAY)    
    
    #if i%2==0:
    raw_data.append(image_to_feature_vector(image))
    raw_labels.append("left")
    #else:
    right = rotate(image, angle=-90)
    raw_data.append(image_to_feature_vector(right))
    raw_labels.append("right")
        
print("Loaded %d images" % len(raw_labels))

Loaded 3214 images


In [61]:
# encode the labels, converting them from strings to integers
le = LabelEncoder()
labels = le.fit_transform(raw_labels)

if False:
    # show data
    for i in range(0,len(raw_data)):
        print("%s"%raw_labels[i])
        showGrey(raw_data[i].reshape(28,28))
        if i>50:
            break
        
# scale the input image pixels to the range [0, 1], then transform
# the labels into vectors in the range [0, num_classes] -- this
# generates a vector for each label where the index of the label
# is set to `1` and all other entries to `0`
data = np.array(raw_data) / 255.0

data = data[:, :, :, np.newaxis]
labels = np_utils.to_categorical(labels, 2)
 
# partition the data into training and testing splits, using 75%
# of the data for training and the remaining 25% for testing
(trainData, testData, trainLabels, testLabels) = train_test_split(
    data, labels, test_size=0.25, random_state=42)
    
print("Train data shape: "+ str(trainData.shape))
print("Test data shape: "+ str(testData.shape))


Train data shape: (2410, 28, 28, 1)
Test data shape: (804, 28, 28, 1)


In [62]:
# network architectures 

class LeNet:
    @staticmethod
    def build(input_shape, classes, weightsPath=None):
        model = Sequential()
        model.add(Conv2D(20, (5, 5), activation='relu', padding="same", input_shape=input_shape))
        model.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2)))
        model.add(Conv2D(50, (5, 5), activation='relu', padding="same"))
        model.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2)))
        model.add(Flatten())
        model.add(Dense(500, activation='relu'))
        model.add(Dense(classes, activation='softmax'))

        if weightsPath is not None:
            model.load_weights(weightsPath)
 
        return model
        
class LeNet2:
    @staticmethod
    def build(input_shape, classes, weightsPath=None):
        model = Sequential()
        model.add(Conv2D(32, (3, 3), activation='relu', padding="same", input_shape=input_shape))
        model.add(Conv2D(64, (3, 3), activation='relu', padding="same"))
        model.add(MaxPooling2D(pool_size=(2, 2)))
        model.add(Dropout(0.25))
        model.add(Flatten())
        model.add(Dense(128, activation='relu'))
        model.add(Dropout(0.5))
        model.add(Dense(classes, activation='softmax'))

        if weightsPath is not None:
            model.load_weights(weightsPath)
 
        return model
    
class LeNet3:
    @staticmethod
    def build(input_shape, classes, weightsPath=None):
        model = Sequential()
        print(input_shape)
        model.add(Conv2D(64, (3, 3), activation='relu', input_shape=input_shape))
        model.add(Conv2D(64, (3, 3), activation='relu'))
        model.add(MaxPooling2D(pool_size=(2, 2)))
        model.add(Dropout(0.25))
        model.add(Flatten())
        model.add(Dense(128, activation='relu'))
        model.add(Dropout(0.5))
        model.add(Dense(classes, activation='softmax'))

        if weightsPath is not None:
            model.load_weights(weightsPath)
 
        return model
    

In [71]:
# define the model
model = LeNet3.build((size[0],size[1],1), classes=2, weightsPath=None)
print(model.summary())

(28, 28, 1)
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_27 (Conv2D)           (None, 26, 26, 64)        640       
_________________________________________________________________
conv2d_28 (Conv2D)           (None, 24, 24, 64)        36928     
_________________________________________________________________
max_pooling2d_22 (MaxPooling (None, 12, 12, 64)        0         
_________________________________________________________________
dropout_11 (Dropout)         (None, 12, 12, 64)        0         
_________________________________________________________________
flatten_14 (Flatten)         (None, 9216)              0         
_________________________________________________________________
dense_27 (Dense)             (None, 128)               1179776   
_________________________________________________________________
dropout_12 (Dropout)         (None, 128)               0        

In [72]:
# train
verbose=1
epochs=20
batch_size=128
opt = SGD(lr=0.01)
#opt = Adadelta()
model.compile(loss=categorical_crossentropy, optimizer=opt, metrics=["accuracy"])
early_stopping = EarlyStopping(monitor='loss', patience=2, verbose=1, mode='auto')
model.fit(trainData, trainLabels, batch_size=batch_size, epochs=epochs, callbacks=[early_stopping], verbose=verbose)

# evaluate
(loss, accuracy) = model.evaluate(testData, testLabels, batch_size=batch_size, verbose=verbose)
print("Test set loss: {:.4f}, accuracy: {:.2f}%".format(loss, accuracy * 100))

# save weights to disk
model_filepath = "/Users/rokickik/Downloads/Workstation/orientation/lenet.weights"
model.save(model_filepath, overwrite=True)

Epoch 1/20
2410/2410 [==============================] - 6s 3ms/step - loss: 0.6101 - acc: 0.8328
Epoch 2/20
2410/2410 [==============================] - 7s 3ms/step - loss: 0.3649 - acc: 0.9946
Epoch 3/20
2410/2410 [==============================] - 8s 3ms/step - loss: 0.1493 - acc: 0.9988
Epoch 4/20
2410/2410 [==============================] - 8s 3ms/step - loss: 0.0681 - acc: 0.9988
Epoch 5/20
2410/2410 [==============================] - 8s 3ms/step - loss: 0.0392 - acc: 0.9988
Epoch 6/20
2410/2410 [==============================] - 8s 3ms/step - loss: 0.0276 - acc: 0.9992
Epoch 7/20
2410/2410 [==============================] - 7s 3ms/step - loss: 0.0224 - acc: 0.9992
Epoch 8/20
2410/2410 [==============================] - 9s 4ms/step - loss: 0.0189 - acc: 0.9992
Epoch 9/20
2410/2410 [==============================] - 9s 4ms/step - loss: 0.0152 - acc: 0.9988
Epoch 10/20
2410/2410 [==============================] - 7s 3ms/step - loss: 0.0148 - acc: 0.9988
Epoch 11/20
2410/2410 [======

In [78]:
# test on actual right-oriented data

from keras.models import load_model

test_dir = "/Users/rokickik/Downloads/Workstation/orientation/20x/right"

# initialize the class labels 
CLASSES = ["left", "right"]
 
# load the network
print("loading model")
model = load_model(model_filepath)

all_correct = True

# loop over our testing images
for imagePath in paths.list_images(test_dir):
    # load the image, resize it to a fixed 32 x 32 pixels (ignoring
    # aspect ratio), and then extract features from it
    filename = imagePath[imagePath.rfind("/") + 1:]
    image = cv2.imread(imagePath)
    image = cv2.cvtColor(image, cv2.COLOR_RGB2GRAY)
    features = image_to_feature_vector(image) / 255.0
    features = features[np.newaxis, :, :, np.newaxis]
    
    probs = model.predict(features)[0]
    prediction = probs.argmax(axis=0)

    if prediction!=1:
        print("INCORRECT PREDICTION for %s" % (filename))
        all_correct = False
    if False:
        # draw the class and probability on the test image and display it
        label = "{}: {:.2f}%".format(CLASSES[prediction], probs[prediction] * 100)
        print("Classified {} as {}".format(filename, label))
        cv2.putText(image, label, (40, 95), cv2.FONT_HERSHEY_SIMPLEX, 3.0, (255, 255, 255), 10)
        showGrey(image)

if all_correct:
    print("All correct")
        

loading model
All correct


In [ ]:
# Not yet working

from sklearn.grid_search import GridSearchCV, RandomizedSearchCV

# construct the set of hyperparameters to tune
#params = {"n_neighbors": np.arange(1, 31, 2),
#          "metric": ["euclidean", "cityblock"]}

#grid = GridSearchCV(model, params)
grid = RandomizedSearchCV(model, params)
start = time.time()
grid.fit(trainData, trainLabels)
print("Grid search took {:.2f} seconds".format(time.time() - start))
acc = grid.score(testData, testLabels)
print("Grid search accuracy: {:.2f}%".format(acc * 100))
print("Grid search best parameters: {}".format(grid.best_params_))

